# (0) Dependencies and Parameters

In [25]:
import numpy as np
import scipy as sp

In [26]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [27]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [28]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation
        # Refer to this source for derivation: https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()

# (1) Finishing the controller
In this notebook, you'll take your previous implementation where we just controller altitude (z-position) and expand it to build the full position-attitude loop.

**Note that I updated the controller params class with some extra parameters!**

#### Recommended steps
- Copy your 3D quadrotor model
- Copy your controller implementation
- Expand the controller class
    - Compute the desired roll/pitch using the equations from the lecture
    - Compute the attitude errors
    - Compute the desired angular acceleration
    - Compute `u_2`
    - Tune your system (the gains provided _should_ provide a relatively stable starting point!)

## (1.1) The dynamics

In [29]:
# NOTE(roy) feel free to copy large parts from your previous assignment!
class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:     ## DONE!
        # NOTE(roy): I advise you to scroll through dynamics.py and see what the __init__
        # function in the base class does!
        ## Sets times steps =.01, and state to the origin
        super().__init__()

    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray) -> np.ndarray:      ## DONE!
        # NOTE(roy): Implement the simple rotor trust model we discussed in the slides
        # To see how complex this can get, see for example: https://arxiv.org/pdf/1601.00733.pdf

        # NOTE(elijah) To turn rotor_rates into thrust we use F = k_T w^2      
        # k_T = Thrust coefficient
        # w = Rotor rate
        k_T = p.k_thrust
        w =  rotor_rates
        F = k_T * np.square(w)
        return F

    def step(self, t: float, input: QuadrotorCommands) -> QuadrotorState:   ## WORK IN PROGRESS
        # NOTE(roy) Use only the first two rotor rates!
        Forces = FullQuadrotorDynamics.rotor_thrust_model(input.rotor_rates)
        L = p.arm_length
        k_t = p.k_thrust
        k_m = p.k_torque
        K = k_m/k_t


        mixing_matrix = np.array([
            [1,1,1,1],
            [0,L,0,-L],
            [-L,0,L,0],
            [K,-K,K,-K]
        ])
        U = mixing_matrix @ Forces

        # NOTE(roy) Implement the thrust/torque calculation we discussed in the slides
        
        u1 = U[0]           # Total Thrust
        u2 = U[1:4]         # Total Torque
        # raise NotImplementedError("Thrust/torque calculation not implemented!")

        # NOTE(roy) this local function is a wrapper around the class state_derivative function         ## DONE!!
        # all this does is convert the our state dataclass from/into a vector that the scipy
        # integration solver can work with. See dynamics.py for the definition of these
        # from/to state vector functions.
        def state_derivative(t: float, state: np.ndarray) -> np.ndarray:
            return FullQuadrotorDynamics.state_derivative(
                QuadrotorState.from_state_vector(state),
                u1,
                u2,
            ).to_state_vector()

        state_vector = self.state.to_state_vector()


        # raise NotImplementedError("Use the scipy function solve_ivp to integrate to the next timestep")
         # NOTE(roy) solution = sp.integrate.solve_ivp(....
        
        solution = sp.integrate.solve_ivp(
            fun= state_derivative,
            t_span = (0, self.dt),
            y0 = state_vector,
        )

        self.state = QuadrotorState.from_state_vector(solution['y'][:, -1])
        return self.state

    @staticmethod
    def state_derivative(state: QuadrotorState, u1: float, u2: np.ndarray) -> QuadrotorState:        ## WORK IN PROGRESS
        # raise NotImplementedError("State derivative not implemented!")

        # NOTE(roy) compute the relevant values here! Note that we're using only the x- and z dimensions for translation
        # and only the y-axis for rotation for this planar model.
        # accel = np.zeros(3)
        g = p.g
        m = p.mass

        R = state.orientation     # Rotation matrix
        accel = np.array([0,0,-g]) +  R * np.array([0,0,u1/m]) # np.array([0,0,-g]) +  R * np.array([0,0,u1/m])

        I = p.inertia
        M = u2
        w = state.angular_velocity
        I_inv = np.linalg.inv(I)
        complicated_sum = M - np.cross(w, np.matmul(I,w))
        angular_accel = np.matmul(I_inv, complicated_sum)

        # NOTE(roy) this might look a little confusing at first glance, but we're populating DERIVATIVES
        # of the state here! So position -> velocity, velocity -> acceleration, etc...
        # If you can think of a more readable way to do this (e.g. defining a QuadrotorStateDerivative class)
        # feel free to open a pull-request into the upstream repository!
        state_derivative = QuadrotorState(
            position=state.velocity,
            velocity=accel,
            orientation=dRot3(state.orientation, state.angular_velocity),
            angular_velocity=angular_accel,
        )

        return state_derivative

## (1.2) The controller

In [30]:
@dataclass
class ControllerParams:
    """The same as above but for the controller gains"""

    # NOTE(roy) diag just means we get a matrix with these values on the diagonal,
    # so these are 3x3 matrices (print them if you're unsure!)
    # We use this formulation because it makes the math a bit more readable later on!
    K_p: np.ndarray = field(default_factory=lambda: np.diag([1, 1, 100]))
    K_d: np.ndarray = field(default_factory=lambda: np.diag([0.5, 0.5, 10]))

    # Added gains for the attitude controller
    Ka_p: np.ndarray = field(default_factory=lambda: np.diag([300, 300, 50]))
    Ka_d: np.ndarray = field(default_factory=lambda: np.diag([50, 50, 20]))

    # Min/max rotor rates!
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600

controller_p = ControllerParams()

In [31]:
yaw = 0
rotation_matrix = np.array(
    [[np.cos(yaw), np.sin(yaw)],
    [np.sin(yaw), -1* np.cos(yaw)]]
    )

rotation_matrix

array([[ 1.,  0.],
       [ 0., -1.]])

In [32]:
class Controller(ControllerBase):
    @staticmethod
    def rotor_rates_from_u(u: np.ndarray) -> np.ndarray:        # NOTE(Elijah) DONE! (Helper function)
        # NOTE(roy) implement your conversion from u (so that's [u1, u2_1, u2_2, u2_3] like we discussed, a 4-sized vector)
        # to rotor rates (which is also a four-sized vector)

        # NOTE(roy) the steps here are:
        #  1. convert u -> forces with the inverse mixing matrix
        L = p.arm_length
        k_t = p.k_thrust
        k_m = p.k_torque
        K = k_t/k_m
        mixing_matrix = np.array([
            [1,1,1,1],
            [0,L,0,-L],
            [-L,0,L,0],
            [K,-K,K,-K]
        ])
        inv_mixing_matrix = np.linalg.inv(mixing_matrix)
        Forces = inv_mixing_matrix @ u

        #  2. convert forces -> rotor rates by inverting the rotor rate model
        Forces = np.maximum(Forces, 0)
        rotor_rates = np.sqrt(Forces / k_t)

        #  3. limit the rotor rates with rotor_rate_min/rotor_rate_max above!
        min_rate = controller_p.rotor_rate_min
        max_rate = controller_p.rotor_rate_max
        for i, rate in enumerate(rotor_rates):
            if rate > max_rate:
                rotor_rates[i] = max_rate
            elif rate < min_rate:
                rotor_rates[i] = min_rate
            else:
                pass
        
        return rotor_rates
        # raise NotImplementedError("rotor_rates_from_u not implemented!")

    def step(self, t: float, trajectory: TrajectoryState, state: QuadrotorState) -> QuadrotorCommands:      # NOTE(Elijah)  NOT DONE!
        desired_position, desired_velocity = trajectory.position, trajectory.velocity
        actual_position, actual_velocity = state.position, state.velocity
        
        # NOTE(roy) first compute the desired acceleration using the equation from the lectures!
        # I already extracted the values you need to compute it above
        e = desired_position - actual_position
        e_dot = desired_velocity - actual_velocity
        K_d = controller_p.K_d
        K_p = controller_p.K_p
        commanded_acceleration = (K_d @ e_dot) + (K_p @ e)  # REMEBER K_D Is a matrix
  
        # NOTE(roy) now compute u_1 from the commanded acceleration using the linearized equation from the lectures!
        m = p.mass 
        # m = p.mass * .5
        g = p.g

        u_1 = m * (g + commanded_acceleration[2])  # Your equation here!


        # (1) Compute desired roll and pitch using equations from the lectures (DONE!)
        x_y_acceleration = commanded_acceleration[0:2]
        yaw = state.orientation.to_yaw_pitch_roll()[0]
        rotation_matrix = np.array( 
            [[np.cos(yaw), np.sin(yaw)],
            [np.sin(yaw), -np.cos(yaw)]]
        )

        rotation_matrix_T = np.transpose(rotation_matrix)
        roll_pitch_vector = rotation_matrix_T @ x_y_acceleration
        des_roll = roll_pitch_vector[1]
        des_pitch = roll_pitch_vector[0]

        
        # (2) Compute attitude errors angle is given by [roll, pitch, yaw]
        des_angle = np.array([des_roll, des_pitch, trajectory.yaw])
        act_angle = np.flip(state.orientation.to_yaw_pitch_roll())
        E = des_angle - act_angle

        des_ang_velocity = np.array([0, 0, trajectory.yaw_rate])
        act_ang_velocity = state.angular_velocity
        E_dot = des_ang_velocity - act_ang_velocity


        # (3) Compute u_2
        K_d = controller_p.Ka_d
        K_p = controller_p.Ka_p
        I = p.inertia

        error_sum = (K_p @ E) +  (K_d @ E_dot) 
        u_2 = I @ error_sum

        # u_2 = np.zeros(3)
        u = np.array([u_1, *u_2])

        rotor_rates = Controller.rotor_rates_from_u(u)
        
       # assert(1==0)
        return QuadrotorCommands(rotor_rates)

### The trajectory
Just something simple, mess around with it!

In [33]:
class JumpTrajectory(TrajectoryBase):
    def eval(self, t: float):
        x = 0 if t < 1.0 or t > 5.0 else 1.0
        y = 0 if t < 2.0 or t > 5.0 else 1.0
        position = np.array([x, y, 0])

        yaw_rate = 0.5
        yaw = yaw_rate * t
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=yaw_rate)
    

class HelicalTrajectory(TrajectoryBase):
    def eval(self, t: float):               # At each point in time (t) this function returns a desired position
        position = np.array([np.cos(t), np.sin(t), .3* t])
        return TrajectoryState(t, position=position)

# class JumpTrajectory(TrajectoryBase):
#     def eval(self, t: float):
#         altitude = 0 if t < 1.0 else 1.0
#         position = np.array([0, 0, altitude])

#         return TrajectoryState(t, position=position)

In [34]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d

In [35]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.005,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(),
        trajectory=HelicalTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [36]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 10.0)

Output()

### Looking at the controller behavior
If all is well, the quadrotor should follow the trajectory!

Try some experiments:
- Can you make the controller more/less damped and faster/slower to respond?
- Can you write a trajectory that has feasible velocities/positions?
    - Hint: the trajectory that's currently there is infeasible, why?

### Commit your updated notebook to your fork!